# Lecture Plan

1. AutoGen

---

### LangChain vs AutoGen


**LangChain:**

1. LLM based applications with Chains, Tools and Agents
2. LangChain natievly supports to multiple external data sources
3. LangChain uses a Chain Based approach. 
4. LangChain recently launched LangGraph which supports DAG in Chains and extends LangChain's existing features to Multi Agent workflows.

**AutoGen**

1. Multi Agent Framework from Microsoft.
2. Used to build LLM Applications with workflows involving the Multi Agent Conversations.
3. AutoGen uses an Agent Based  Approach
4. AutoGen recently launched AutoGen studio. It is a web interface for rapid prototyping.

![AutoGen Framework](./images/autogen.png)


In [ ]:
! pip install pyautogen

#### Config List

- specify the model and other configuration info to agent.

**ways to generate config list:**

1. `get_config_list`: generates config list for API calls
2. `config_list_openai_aoai`:  generates config list for OpenAI and Azure OpenAI endpoints
3. `config_list_from_json`: loads configuration from a JSON file. `filter_dict` is used to filter the `config_list_from_json` file.
4. `config_list_from_models`: creates configurations based on provided list of models.
5. `config_list_from_dotenv`: loads configuration from a .env file. `filter_dict` is used to filter the `config_list_from_json` file.

In [ ]:
import autogen

In [1]:
from autogen import config_list_from_dotenv

config_list = config_list_from_dotenv(
    dotenv_file_path='C:/TrainingMaterial/generative-ai/genai-material/.env',
    filter_dict={
        "model":{
            "gpt-3.5-turbo"
        }
    }
)

#### Agents

AutoGen provides an abstraction for creating the agents with following features:

- **Conversable:** Any agent can send or recieve messages from the other agent.
- **Customizable:** Agents in autogen can be integrated with LLMs, tools and humans.

#### Multi Agent Conversational Framework

![AutoGen Agents](./images/autogen_agents.png)


1. `ConversableAgent`: Genric class for agents that converse with each other by exchnaging messages.
2. `AssistantAgent`: designed to act as an AI Assistant using LLMs.
3. `UserProxyAgent`: proxy for humans.
4. `GroupChatManager` : A chat manager that can manage a group chat of multiple agents.

In [22]:
# Code generation agent

from autogen import AssistantAgent, UserProxyAgent

assistant = AssistantAgent(
    name = "assistant",
    llm_config={
        "config_list": config_list,
        "temperature": 0.0,
        "cache_seed": 42

    },
    code_execution_config={
        "use_docker" : False
    },
    max_consecutive_auto_reply= 10
)


user_proxy = UserProxyAgent(

    name = "user_proxy",
    system_message="reply TERMINATE when you are done with the code",
    human_input_mode="TERMINATE",
    code_execution_config={
        "work_dir": "C:/TrainingMaterial/generative-ai/genai-material/trng-1855/trng-git-repo/trng-1855/week-7/",
        "use_docker" : False
    },
    llm_config={
        "config_list": config_list,
        "temperature": 0.0,
        "cache_seed": 42

    },
    is_termination_msg = lambda x : x.get("content", "").rstrip().endswith("TERMINATE"),
    max_consecutive_auto_reply= 10
    
)

# response = user_proxy.initiate_chat(
#     assistant,
#     message = """Create a Langchain agent with RAG""",
#     summry_method="reflection_from_llms"
# )






In [23]:
product_manager = AssistantAgent(
    name = "product_manager",
    llm_config={
        "config_list": config_list,
        "temperature": 0.0,
        "cache_seed": 42

    },
    code_execution_config={
        "use_docker" : False
    },
    max_consecutive_auto_reply= 10,
    system_message= "Agent to ask unique coding questions as a continuation for user questions"    
)

In [24]:
# Agent Group Chat

from autogen import GroupChat, GroupChatManager


agent_group = GroupChat(agents=[assistant, user_proxy, product_manager], messages = [], max_round = 10)

manager = GroupChatManager(
    groupchat= agent_group,
    max_consecutive_auto_reply= 10,
    llm_config={
        "config_list": config_list,
        "temperature": 0.0,
        "cache_seed": 42

    },
    code_execution_config={
        "use_docker":False
        
    }
)

result = user_proxy.initiate_chat(
    manager,
    message = "Code to search a number in a list?",
)



user_proxy (to chat_manager):

Code to search a number in a list?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...


user_proxy (to chat_manager):

Sure! Here's an example code to search for a number in a list:

```python
def search_number(numbers, target):
    for number in numbers:
        if number == target:
            return True
    return False

# Example usage
numbers = [1, 2, 3, 4, 5]
target = 3

if search_number(numbers, target):
    print(f"The number {target} is found in the list.")
else:
    print(f"The number {target} is not found in the list.")
```

In this code, the `search_number` function takes in a list of numbers and a target number to search for. It iterates through each number in the list and checks if it matches the target number. If a match is found, it returns `True`. If no match is found after iterating through the entire list, it returns `False`.

In the example usage, we define a list of numbers `[1, 2, 3, 4, 5]` and a target number `3`. We call the `search_number` function with these arguments and check the returned value. If the number is found, it prints a message indi

product_manager (to chat_manager):

exitcode: 0 (execution succeeded)
Code output: 
The number 3 is found in the list.


--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
user_proxy (to chat_manager):

Great! I'm glad the code worked for you. If you have any more questions or need further assistance, feel free to ask.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
user_proxy (to chat_manager):

TERMINATE

--------------------------------------------------------------------------------


#### References


- [AutoGen Docs](https://microsoft.github.io/autogen/docs/Getting-Started)
- [AutoGen SDK](https://microsoft.github.io/autogen/docs/reference/agentchat/conversable_agent)